# 서울시 범죄율
## 주민들이 거주하는 구는 안전할까?

1. 라이브러리 호출
2. 데이터 불러오기
3. 구글맵 API 호출
4. 데이터 전처리


- crime_anal_police 데이터셋
- station_address = [] # 경찰서 주소
- station_lat = [] # 위도 latitude
- station_lng = [] #  경도 longitude


## 1. 라이브러리

In [106]:
import numpy as np   # 1.16.5
import pandas as pd  # 0.25.1
import googlemaps    # 4.6.0  (위도,경도) 경찰서의 위도 경도 받기 위해
from sklearn import preprocessing # '0.21.3' 데이터 '정규화 스케일'조정

%matplotlib inline
import matplotlib.pyplot as plt # 3.1.1
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False
f_path = 'C:\Windows\Fonts\malgun.ttf'
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font',family=font_name)

## 2. 데이터 불러오기

In [4]:
crime_anal_police = pd.read_csv('~~\\02. crime_in_Seoul.csv'
                                , thousands=','
                                , encoding='euc-kr')


In [7]:
crime_anal_police.shape # (31,11)

(31, 11)

In [8]:
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170
1,종로서,3,3,6,5,115,98,1070,413,1278,1070
2,남대문서,1,0,6,4,65,46,1153,382,869,794
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711
4,혜화서,3,2,5,4,96,63,1114,424,1015,861


## 3. 구글맵 API 불러오기

In [27]:
gmaps_key="~~~~~~~~" # google map API key
gmaps = googlemaps.Client(key=gmaps_key)

In [28]:
gmaps.geocode('서울중부경찰서', language='ko')

[{'address_components': [{'long_name': '２７',
    'short_name': '２７',
    'types': ['premise']},
   {'long_name': '수표로',
    'short_name': '수표로',
    'types': ['political', 'sublocality', 'sublocality_level_4']},
   {'long_name': '중구',
    'short_name': '중구',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': '서울특별시',
    'short_name': '서울특별시',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': '대한민국',
    'short_name': 'KR',
    'types': ['country', 'political']},
   {'long_name': '100-032',
    'short_name': '100-032',
    'types': ['postal_code']}],
  'formatted_address': '대한민국 서울특별시 중구 수표로 27',
  'geometry': {'location': {'lat': 37.56361709999999, 'lng': 126.9896517},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 37.5649660802915,
     'lng': 126.9910006802915},
    'southwest': {'lat': 37.5622681197085, 'lng': 126.9883027197085}}},
  'partial_match': True,
  'place_id': 'ChIJc-9q5uSifDURLhQmr5wkXmc',
 

## 4. 데이터 전처리

### 4-1. 경찰서 주소, 위도, 경도 추출

In [42]:
tmp = gmaps.geocode('서울중부경찰서', language='ko')
print(tmp[0].get('formatted_address')) # 대한민국 서울특별시 중구 수표로 27
tmp[0].keys()

대한민국 서울특별시 중구 수표로 27


dict_keys(['address_components', 'formatted_address', 'geometry', 'partial_match', 'place_id', 'plus_code', 'types'])

In [75]:
station_name = []
for name in crime_anal_police['관서명']:
    station_name.append(''.join(['서울', name[:-1], '경찰서']))
station_name

['서울중부경찰서',
 '서울종로경찰서',
 '서울남대문경찰서',
 '서울서대문경찰서',
 '서울혜화경찰서',
 '서울용산경찰서',
 '서울성북경찰서',
 '서울동대문경찰서',
 '서울마포경찰서',
 '서울영등포경찰서',
 '서울성동경찰서',
 '서울동작경찰서',
 '서울광진경찰서',
 '서울서부경찰서',
 '서울강북경찰서',
 '서울금천경찰서',
 '서울중랑경찰서',
 '서울강남경찰서',
 '서울관악경찰서',
 '서울강서경찰서',
 '서울강동경찰서',
 '서울종암경찰서',
 '서울구로경찰서',
 '서울서초경찰서',
 '서울양천경찰서',
 '서울송파경찰서',
 '서울노원경찰서',
 '서울방배경찰서',
 '서울은평경찰서',
 '서울도봉경찰서',
 '서울수서경찰서']

In [76]:
station_address = []
station_lat = [] # 위도 latitude
station_lng = [] #  경도 longitude
i = 0
for name in station_name:
    tmp = gmaps.geocode(name, language='ko')
    station_address.append(tmp[0].get('formatted_address'))
    
    tmp_loc = tmp[0].get("geometry")
    station_lat.append(tmp_loc['location']['lat'])
    station_lng.append(tmp_loc['location']['lng'])
    
    print(f"{name} -> {station_address[i]}")
    i += 1

서울중부경찰서 -> 대한민국 서울특별시 중구 수표로 27
서울종로경찰서 -> 대한민국 서울특별시 종로구 율곡로 46
서울남대문경찰서 -> 대한민국 서울특별시 중구 한강대로 410
서울서대문경찰서 -> 대한민국 서울특별시 서대문구 통일로 113
서울혜화경찰서 -> 대한민국 서울특별시 종로구 창경궁로 112-16
서울용산경찰서 -> 대한민국 서울특별시 용산구 백범로 329
서울성북경찰서 -> 대한민국 서울특별시 성북구 삼선동 보문로 170
서울동대문경찰서 -> 대한민국 서울특별시 동대문구 약령시로21길 29
서울마포경찰서 -> 대한민국 서울특별시 마포구 마포대로 183
서울영등포경찰서 -> 대한민국 서울특별시 영등포구 국회대로 608
서울성동경찰서 -> 대한민국 서울특별시 성동구 행당동 왕십리광장로 9
서울동작경찰서 -> 대한민국 서울특별시 동작구 노량진로 148
서울광진경찰서 -> 대한민국 서울특별시 광진구 구의동 자양로 167
서울서부경찰서 -> 대한민국 서울특별시 은평구 녹번동 177-15
서울강북경찰서 -> 대한민국 서울특별시 강북구 오패산로 406
서울금천경찰서 -> 대한민국 서울특별시 금천구 시흥대로73길 50
서울중랑경찰서 -> 대한민국 서울특별시 중랑구 묵2동 249-2
서울강남경찰서 -> 대한민국 서울특별시 강남구 테헤란로114길 11
서울관악경찰서 -> 대한민국 서울특별시 관악구 관악로5길 33
서울강서경찰서 -> 대한민국 서울특별시 강서구 화곡6동 980-27
서울강동경찰서 -> 대한민국 서울특별시 강동구 성내로 57
서울종암경찰서 -> 대한민국 서울특별시 성북구 화랑로7길 32
서울구로경찰서 -> 대한민국 서울특별시 구로구 가마산로 235
서울서초경찰서 -> 대한민국 서울특별시 서초구 서초3동 반포대로 179
서울양천경찰서 -> 대한민국 서울특별시 양천구 목동동로 99
서울송파경찰서 -> 대한민국 서울특별시 송파구 중대로 221
서울노원경찰서 -> 대한민국 서울특별시 노원구 노원로 283
서울방배경찰서 -> 대한민국 서울특별시 서초구 동작대로

In [77]:
gu_name = []

for name in station_address:
    tmp = name.split()
    
    tmp_gu = [gu for gu in tmp if gu[-1] == '구'][0]
    
    gu_name.append(tmp_gu)
    
crime_anal_police['구별'] = gu_name
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,중구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,종로구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,중구
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,서대문구
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,종로구


In [78]:
crime_anal_police[crime_anal_police['관서명']=='금천서']

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
15,금천서,3,4,6,6,151,122,1567,888,2054,1776,금천구


#### 임시저장

In [80]:
crime_anal_police.to_csv('~~~\\Crime in Seoul.csv'
                        , sep=','
                        , encoding='cp949')

### 4-2.범죄 데이터 구별로 정리

In [82]:
crime_anal_raw = pd.read_csv('~~~\\02. crime_in_Seoul_include_gu_name.csv'
                            ,encoding="utf-8")
crime_anal_raw.shape # (31,13)
crime_anal_raw.head(3)

,Unnamed: 0,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,0,중부서,2,2,3,2,105,65,1395,477,1355,1170,중구
1,1,종로서,3,3,6,5,115,98,1070,413,1278,1070,종로구
2,2,남대문서,1,0,6,4,65,46,1153,382,869,794,중구


In [85]:
crime_anal_raw = pd.read_csv('~~\\02. crime_in_Seoul_include_gu_name.csv'
                            ,encoding="utf-8"
                            ,index_col=0) # 첫번째 열을 index로 지정
crime_anal_raw.head(3)

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,중구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,종로구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,중구


    - pivot_table사용 -> 구별로 범죄율 볼꺼니까 

In [91]:
crime_anal_ori = pd.pivot_table(crime_anal_raw, index='구별', aggfunc=np.sum)
crime_anal = crime_anal_ori.copy() # id(crime_anal) not is id(crime_anal_ori) = True
crime_anal.head(3)

,강간 검거,강간 발생,강도 검거,강도 발생,살인 검거,살인 발생,절도 검거,절도 발생,폭력 검거,폭력 발생
구별,,,,,,,,,,
강남구,349,449,18,21,10,13,1650,3850,3705,4284
강동구,123,156,8,6,3,4,789,2366,2248,2712
강북구,126,153,13,14,8,7,618,1434,2348,2649


In [93]:
crime_anal['강간검거율'] = crime_anal['강간 검거']/crime_anal['강간 발생']*100
crime_anal['강도검거율'] = crime_anal['강도 검거']/crime_anal['강도 발생']*100
crime_anal['살인검거율'] = crime_anal['살인 검거']/crime_anal['살인 발생']*100
crime_anal['절도검거율'] = crime_anal['절도 검거']/crime_anal['절도 발생']*100
crime_anal['폭력검거율'] = crime_anal['폭력 검거']/crime_anal['폭력 발생']*100

del crime_anal['강간 검거']
del crime_anal['강도 검거']
del crime_anal['살인 검거']
del crime_anal['절도 검거']
del crime_anal['폭력 검거']

crime_anal.head()

,강간 발생,강도 발생,살인 발생,절도 발생,폭력 발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,449,21,13,3850,4284,77.728285,85.714286,76.923077,42.857143,86.484594
강동구,156,6,4,2366,2712,78.846154,133.333333,75.000000,33.347422,82.890855
강북구,153,14,7,1434,2649,82.352941,92.857143,114.285714,43.096234,88.637222
관악구,320,12,9,2706,3298,69.062500,116.666667,88.888889,30.561715,80.109157
광진구,240,14,4,3026,2625,91.666667,185.714286,100.000000,42.200925,83.047619


In [104]:
con_list = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']

for column in con_list:
    crime_anal.loc[crime_anal[column] > 100, column] = 100 # 100이넘는애들의 해당데이터값을 100으로
    
crime_anal.head()

,강간 발생,강도 발생,살인 발생,절도 발생,폭력 발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,449,21,13,3850,4284,77.728285,85.714286,76.923077,42.857143,86.484594
강동구,156,6,4,2366,2712,78.846154,100.000000,75.000000,33.347422,82.890855
강북구,153,14,7,1434,2649,82.352941,92.857143,100.000000,43.096234,88.637222
관악구,320,12,9,2706,3298,69.062500,100.000000,88.888889,30.561715,80.109157
광진구,240,14,4,3026,2625,91.666667,100.000000,100.000000,42.200925,83.047619


In [105]:
crime_anal.rename(columns={
    '강간 발생' : '강간'
    ,'강도 발생' : '강도'
    ,'살인 발생' : '살인'
    ,'절도 발생' : '절도'
    ,'폭력 발생' : '폭력'}, inplace=True)
crime_anal.head(3)

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,449,21,13,3850,4284,77.728285,85.714286,76.923077,42.857143,86.484594
강동구,156,6,4,2366,2712,78.846154,100.000000,75.000000,33.347422,82.890855
강북구,153,14,7,1434,2649,82.352941,92.857143,100.000000,43.096234,88.637222


    - sklearn -> 데이터 '정규화 스케일링'조정

In [116]:
col = ['강간','강도','살인','절도','폭력']
# min-max scaler : 모든 특성이 0~1 사이에 위치하도록
x = crime_anal[col].values
min_max_scaler = preprocessing.MinMaxScaler()

# fit_transform = fit + tansform 2과정을 한번에 수행
x_scaled = min_max_scaler.fit_transform(x.astype(float)) # 정규화
crime_anal_norm = pd.DataFrame(x_scaled, columns=col, index=crime_anal.index)

col2 = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']
crime_anal_norm[col2] = crime_anal[col2] # 정규화 외 feature 이어붙이기
crime_anal_norm.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,1.000000,0.941176,0.916667,0.953472,0.661386,77.728285,85.714286,76.923077,42.857143,86.484594
강동구,0.155620,0.058824,0.166667,0.445775,0.289667,78.846154,100.000000,75.000000,33.347422,82.890855
강북구,0.146974,0.529412,0.416667,0.126924,0.274769,82.352941,92.857143,100.000000,43.096234,88.637222
관악구,0.628242,0.411765,0.583333,0.562094,0.428234,69.062500,100.000000,88.888889,30.561715,80.109157
광진구,0.397695,0.529412,0.166667,0.671570,0.269094,91.666667,100.000000,100.000000,42.200925,83.047619
